<a href="https://colab.research.google.com/github/kashishkhera44/comp215/blob/main/labs/lab02-review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

COMP 215 - LAB 2
----------------
#### Name(s):Kashish Khera
#### Date: 15-01-2025

By the end of this lab you should be able to:
  * use strings, tuples, lists, and dictionaries (review)
  * use *f-string* to simplify string formatting operations
  * write functions and simple unit tests (review)
  * use *list comprehension* to implement map and filter algorithms


During this lab, you will be introduced to the following:
  * API Keys
  * JSON data formats
  * the `datetime` module to represent dates and times
  * the `pprint` module to format data structures

## Near Earth Objects

In this lab, we'll answer some questions about [Near Earth Objects](https://cneos.jpl.nasa.gov/) using NASA's API:  [https://api.nasa.gov/](https://api.nasa.gov/#NeoWS).  But, as usual, let's do the imports first.


In [2]:
import datetime, json, requests
from pprint import pprint    # Pretty Print - built-in python function to nicely format data structures

### dates & f-strings

First we need a short tutorial on python dates and [f-strings](https://realpython.com/python-f-strings/)...

In [3]:
today = datetime.date.today()   # get a date object representing today's date
print(today, type(today))
formatted_date = f'Today is: {today}'   # A format string - notice how the variable `today` is formatted into the string
print(formatted_date)

2025-01-15 <class 'datetime.date'>
Today is: 2025-01-15


### Make an API Query

Let's get some data from the NEO database; here's a query that gets the observation "feed" for today.  Notice the first line of the block is an 'API key'.  An API key is a unique identifier that authenticates a user when making a request to an API.  Run the block below using the API key 'DEMO_KEY' (just to be sure it works), then go to https://api.nasa.gov and get your own API key.  Use your own API key for the remainder of the lab.

In [27]:
API_KEY = 'aMyvm07dkmDrH0Lktiwf5JUg7srNobWQ5BkdVYOY'  # substitute your API key here

# get today's date (as a string)
#today = str(datetime.date.today())
today = '2023-09-15'

# use an f-string to "format" the date and API key varaibles.
url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={today}&end_date={today}&api_key={API_KEY}'

# make the API request (recall from lab 1)
response = requests.request("GET", url, headers={}, data={})

# convert the response to json format (recall from lab 1)
data = json.loads(response.text)

# TIP: print(data) to see the whole data structure returned, here we grab just the list of NEO's:
n_results = data['element_count']
neos = data['near_earth_objects'][today]
print(f'{n_results} Near Earth Objects found for {today}')
pprint(neos[:1])

24 Near Earth Objects found for 2023-09-15
[{'absolute_magnitude_h': 21.57,
  'close_approach_data': [{'close_approach_date': '2023-09-15',
                           'close_approach_date_full': '2023-Sep-15 14:27',
                           'epoch_date_close_approach': 1694788020000,
                           'miss_distance': {'astronomical': '0.297640393',
                                             'kilometers': '44526368.81876291',
                                             'lunar': '115.782112877',
                                             'miles': '27667402.629471758'},
                           'orbiting_body': 'Earth',
                           'relative_velocity': {'kilometers_per_hour': '70213.4577496004',
                                                 'kilometers_per_second': '19.5037382638',
                                                 'miles_per_hour': '43627.9121222448'}}],
  'estimated_diameter': {'feet': {'estimated_diameter_max': 946.2916981006,
       

Next we extract just the potentially hazerdous asteroids, using a Comp115-style list accumulator *loop*:

In [5]:
hazards =  []
for item in neos:
  if item['is_potentially_hazardous_asteroid'] is True:
    hazards.append(item)
print(f'{len(hazards)} potentially hazardous asteroids identified.')

0 potentially hazardous asteroids identified.


### Fetch Complete Data for One Asteroid

Notice that the record for each `neo` is a dictionary with `id` field that uniquely identifies this record in the database.  We can use this `id` to fetch complete orbital and close approach data for the NEO.  For example, this query fetches the complete data set for the first hazardous asteroid...  Notice that the `miss_distance` field contains the distance (in various units) by which the NEO missed an "orbiting body".


In [6]:
id = hazards[0]['id']
url = f'https://api.nasa.gov/neo/rest/v1/neo/{id}?api_key={API_KEY}'
response = requests.request("GET", url, headers={}, data={})
data = json.loads(response.text)

pprint(data)

IndexError: list index out of range

## Exercise 1

In the code cell below, **re-write the accumulator loop** that creates the list of hazards as a [list comprehension](https://realpython.com/lessons/list-comprehensions-overview/) that implements a ["filter"](https://youtu.be/hUes6y2b--0)
Notice how this provides a concise way to "filter" items of interest from a larger data set.

In [7]:
# Ex. 1 your code here

hazards =[item for item in neos if item['is_potentially_hazardous_asteroid'] ]
print(f'{len(hazards)} potentially hazardous asteroids identified. ')

0 potentially hazardous asteroids identified. 




## Exercise 2

In the code cell below, write a python function that takes a list of "close approach data" as a parameter,
and returns a 2-tuple with the (date, miss km) of the closest approach to Earth in the list (where "miss km" is the miss distance in km).

Hints:
* notice the input is a list of dictionaries.  Each dictionary has a 'close_approach_date", "orbiting_body", and 'miss_distance' field.
* we are only interested in the closest approach to "Earth"


Add at least one unit test to check your work - note the test data only needs dictionaries with the fields your function actually uses.


In [28]:
def closest_approach(close_approach_data):
    """
    Finds the closest approach to Earth in the given data.
    """
    earth_keyword = 'earth'

    # Filter Earth approaches with extra cleaning
    earth_approaches = [
        (entry['close_approach_date'], float(entry['miss_distance']))
        for entry in close_approach_data
        if entry.get('orbiting_body', '').strip().lower() == earth_keyword
        and 'close_approach_date' in entry
        and 'miss_distance' in entry
    ]

    # Print the filtered list for debugging
    print("Filtered Earth Approaches:", earth_approaches)

    # Return the closest approach
    if not earth_approaches:
      return None

    # Initialize the closest approach with the first entry
    closest_approach = earth_approaches[0]

    # Loop through each approach to find the closest one
    for approach in earth_approaches[1:]:
        if approach[1] < closest_approach[1]:  # Compare miss distances
            closest_approach = approach

    return closest_approach

#Unit Test
neos = [
    {'orbiting_body': ' Earth ', 'close_approach_date': '2025-01-01', 'miss_distance': '384400'},
    {'orbiting_body': 'Mars', 'close_approach_date': '2025-02-01', 'miss_distance': '54500000'},
    {'orbiting_body': 'earth', 'close_approach_date': '2025-03-01', 'miss_distance': '35900000'}
]


result = closest_approach(neos)

if result:
    closest_date, closest_distance = result
    print(f"Closest Approach: Date: {closest_date}, Distance: {closest_distance} km")
else:
    print("No Earth approaches found.")


Filtered Earth Approaches: [('2025-01-01', 384400.0), ('2025-03-01', 35900000.0)]
Closest Approach: Date: 2025-01-01, Distance: 384400.0 km


## Challenge Exercise

In the code cell below, write a complete program that:
 1. fetches the list of NEO's for this week.
 2. for each NEO, fetch it's complete orbital data and determine its closest approach to Earth
 3. identify which NEO from this week's data makes the closet approach to earth
 4. print a nice message with information about the NEO, which it will approach the Earth, and how close it will come.

Hints:
* you'll need the start and end date - end date is today, see if you can use a [`timedelta`](https://docs.python.org/3/library/datetime.html#timedelta-objects) object to ge the start date (you can do basic "date math" with `timedelta` and `date` objects!)
* you may need to modify the function we wrote in Ex. 2 to return a triple with the NEO's id included;
* lots of opportunity here for more practice with list comprehensions


In [42]:
import requests
from datetime import datetime, timedelta

# Get today's date and calculate the start date (7 days ago)
end_date = datetime.today().date()
start_date = end_date - timedelta(days=7)

# API endpoint to fetch NEOs for the given date range
url = f"https://api.nasa.gov/neo/rest/v1/neo/browse?start_date={start_date}&end_date={end_date}&api_key=aMyvm07dkmDrH0Lktiwf5JUg7srNobWQ5BkdVYOY"

# Fetch the data
response = requests.get(url)
data = response.json()

# List comprehension to gather NEO information
neo_info = [
    (neo['name'], closest_approach['close_approach_date'], closest_approach['miss_distance']['kilometers'], neo['id'])
    for neo in data['near_earth_objects']
    if 'close_approach_data' in neo and neo['close_approach_data']
    for closest_approach in neo['close_approach_data']
]

# Check if there are any NEOs with close approach data
if neo_info:
    # Identify the NEO that makes the closest approach to Earth
    closest_neo = min(neo_info, key=lambda x: float(x[2]))

    # Extract information about the closest NEO
    name, approach_date, distance, neo_id = closest_neo

    # Print a nice message
    print(f"The NEO '{name}' will make its closest approach to Earth on {approach_date}.")
    print(f"It will come as close as {distance} kilometers.")
else:
    print("No NEOs with close approach data found for this week.")


The NEO '1862 Apollo (1932 HA)' will make its closest approach to Earth on 2075-05-17.
It will come as close as 1242388.02026737 kilometers.
